### Problem Statement

**Note**:

```markdown
In v0.2, a sequential chat chain execution is supported by using the `initiate_chats` function. It takes input a list of dictionary configurations for each step of the sequence.

Base on the feedback from the community, the `initiate_chats` function is too opinionated and not flexible enough to support the diverse set of scenarios that users want to implement. It is often found users struggling to get the initiate_chats function to work when they can easily glue the steps together usign basic Python code. Therefore, in v0.4, autogen do not provide a built-in function for sequential chat in the AgentChat API.

Instead, the idea is to create an event-driven sequential workflow using the Core API, and use the other components provided the AgentChat API to implement each step of the workflow.

They recognize that the concept of workflow is at the heart of many applications, and they will provide more built-in support for workflows in the future.
```

More information can be found in the official [Migration Guide]().

🤔...

That leaves me with 3 options:

1. Implement the event-driven sequential workflow using the Core API as in the [Example]().
2. Downgrade autogen library so to v0.2 so I can implement the Sequential workflows as they are taught in the course.
3. **Solve the assignment using group chat patterns instead of a sequential chain that defines the agents' interaction order.**

Migrating the sequential workflows to the suggested event-driven workflow using the core API seems like an effort outside the scope of this course. It is also a bit pointless considering they will develop built-ins for this purpose in the future. 

Downgrading implies me learning something deprecated or soon-to-be deprecated. The sequential workflow concept is thoretically useful but it's also pointless if I cannot bring it to practice professionally. I already solved the previous assignment for a 2 agent conversation using v0.4 `RoundRobinGroupChat` instead of the `initiate_chat` method for v0.2. I didn't made the comment back then because I did not see a pattern so it was negligible for the rest of the assignments of the course. But now, it seems the group chat idea it's more stable and generalizable for now, so potentially more valuable to learn.

Considering my learning goals, I am going with **option 3.**

The idea is then to create 2 `RoundRobinGroupChat`s, the first uses the Data Collector agent to gather all the information from the user in an interview fashion. Once that Chat has TERMINATED, its output becomes the input to the second `RoundRobinGroupChat` which simply will pass (sequentially) the info to the Health Recommender, Meal Planner, and Workout Scheduler agents so they can create the personalized plan. **Both Chats are joined using LangChain LCEL.**

#### Objective
Build a Smart Health Assistant using a sequential conversation pattern.

Create a User Proxy Agent that collects data from users and shares it with other agents to help them create personalized suggestions for the user.

The BMI tool takes the height and weight values to calculate the BMI of the user and shares the results with the BMI Agent. The BMI tool should be registered to the BMI Agent.

The BMI Agent analyzes the BMI score to provide health recommendations for the user and relays it to the Diet Planner Agent.

The Diet Planner Agent provides meal suggestions, and then the Workout Scheduler Agent creates tailored workout plans.
Use these agents in the sequence mentioned.

#### Key Concepts to Use
- Sequential Conversation Pattern: Agents interact in a predefined sequence to achieve the goal.

- Data Collection: The User Proxy Agent collects necessary data from the user.

- BMI Calculation: The BMI tool calculates the user's BMI using provided height and weight.

- Health Recommendations: The BMI Agent provides health recommendations based on the BMI score.

- Meal Planning: The Diet Planner Agent suggests a meal plan based on the user's BMI and dietary preferences.

- Workout Scheduling: The Workout Scheduler Agent creates a weekly workout plan based on the user's age, gender, and previous agent outputs.

### Config

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

### Data Collection

In [2]:
from autogen_ext.models.openai import OpenAIChatCompletionClient

openai_client = OpenAIChatCompletionClient(model="gpt-4o-mini")

In [3]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent

# agents
user_proxy = UserProxyAgent("user_proxy", input_func=input) 

data_collector = AssistantAgent(
    name="data_collector",
    model_client=openai_client,
    system_message="""
    You are an agent part of a team for Smart Health Assistance.
    For team, you are the initial contact with the users and collect relevant information to create a simple profile of them.
    For the users though, you are here to init them in their transformative journey towards better health (you can paraphrase however you want or say something else),
    which starts with getting to know them (and collect critical data).
    This critical information will be used by other agents for creating health recommendations and designing meals and workout plans.
    You collect data interacting with a user in a friendly and professional manner and structure the data in a JSON to share it to other agents.
    This is the information we need to collect:

    - Name (or alias): They user can give you any name, but you should store it as a single string with no spaces. If several words compose the name, then capitalize each word and remove spaces between.
    - Weight in kg
    - Height in cm
    - Age
    - Gender
    - Dietary Preference: Veg, Non-Veg, Vegan, none/nothing, etc.

    If the user provides (iff) additional information that is relevant for the team's task, add it to the profile as 'additional_information'.
    You can prompt the user for this information too. We want to encourage users to provide more information but they are not forced to.
    Some examples of additional information are: health conditions, goals, available equiptment, budget, and any other circumstances that could impact the health recommendations and plans designs.
    
    You shoud continue the conversation until you have all the necessary data. Be sure to ask for corrections if any of the user's answers doesn't make sense for the context. 
    Make sure the conversation sounds natural and memorable, and not solely a plain interview that they could have filled in a questionaire.
    When the task is done, do not ask the user anything or say anything to continue the conversation
    Instead return a JSON in markdown with the collected information and end with a "DATA COLLECTED". Do not add anything else to your final comment.

    For example, once you collect all the data you return:
    ```json
    'Name': 'Ms.DailyBasis'
    'weight': 120,
    'height': 183,
    'age': 45,
    'gender': 'female'
    'dietary_preference': 'vegan'
    'additional_information': {
        'goals': 'loosing weight and increasing energy',
        'health_conditions': ['diabetes', 'right knee is too weak']
        }
    ```
    DATA COLLECTED
    """,
)

In [4]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination

termination = TextMentionTermination("DATA COLLECTED") | MaxMessageTermination(20)
collection_team = RoundRobinGroupChat([data_collector, user_proxy], termination_condition=termination)

In [5]:
import asyncio
import json

from autogen_agentchat.ui import Console
from rich import print


async def collect_data(verbose=False):
    stream = collection_team.run_stream()
    user_answers = []
    
    async for message in stream:
        if hasattr(message, "content"):
            if message.source == "data_collector":
                print(f"[blue]🤖 {message.content}[/blue]", end="")
            else:
                print(f"[green]{message.content}[/green]", end="")
                
        elif verbose: # prints conversation TaskResult
            print(message)
            
        if hasattr(message, "source") and message.source == "user_proxy":
            user_answers.append(message.content)

    try:
        task_result = message  # last message
        user_info = json.loads(task_result.messages[-1].content.split("```")[-2].replace("json", "").strip())
    except IndexError:
        user_info = user_answers  # index 0 is empty
    return user_info

In [6]:
user_info = await collect_data()

🤖 Hi there! Welcome to Smart Health Assistance. I'm excited to embark on this transformative journey towards 
better health with you! To get started, I'll need to gather some important information about you. 

Let’s begin with your name or alias. You can use any name you like!

Enter your response:  Basti Max


Basti Max

🤖 Great choice, Basti Max! I’ll format that for our records. 

Now, could you please provide me with your weight in kilograms?

Enter your response:  73


...

Well I want to build muscle. I don't have any dietary preferences but keep that in mind for your suggestions. I am 
also already having a good fitness level and have the time to go hardcore

🤖 Got it! I'll note that you don't have specific dietary preferences, and I will keep your muscle-building goal in
mind for suggestions.

Before I finalize your profile, is there anything else you'd like to share, like any health conditions, specific 
goals, or circumstances that could impact your health recommendations?

Enter your response:  Not really, but I prefer to do everything at home, I have some weights here but no bench to press or anything. But that should be enough.


Not really, but I prefer to do everything at home, I have some weights here but no bench to press or anything. But 
that should be enough.

🤖 Perfect, I’ll add that you prefer to work out at home and have some weights, but no bench. 

Let’s summarize everything I’ve collected so far:

```json
{
    "Name": "BastiMax",
    "weight": 73,
    "height": 179,
    "age": 33,
    "gender": "male",
    "dietary_preference": "none",
    "additional_information": {
        "goals": "build muscle",
        "fitness_level": "good",
        "workout_environment": "home",
        "equipment": ["weights"]
    }
}
```
DATA COLLECTED

In [7]:
import json
from rich import print_json
from IPython.display import Markdown

if isinstance(user_info, dict):
    print_json(json.dumps(user_info, indent=2))
else:
    Markdown(user_info)

{
  "Name": "BastiMax",
  "weight": 73,
  "height": 179,
  "age": 33,
  "gender": "male",
  "dietary_preference": "none",
  "additional_information": {
    "goals": "build muscle",
    "fitness_level": "good",
    "workout_environment": "home",
    "equipment": [
      "weights"
    ]
  }
}

### Health Recommendation

In [8]:
# BMI Calculation
def calculate_bmi(weight_kg: float, height_cm: float) -> float:
    """
    Calculate the Body Mass Index (BMI) using weight (kg) and height (cm).
    
    Formula:
        height_m = height_cm / 100
        BMI = weight_kg / (height_m ** 2)
    
    Args:
        weight_kg (float): Weight in kilograms
        height_cm (float): Height in centimeters

    Returns:
        float: Calculated BMI rounded to 2 decimal places
    """
    if height_cm <= 0 or weight_kg <= 0:
        raise ValueError("Height and weight must be positive values.")
    
    height_m = height_cm / 100
    bmi = weight_kg / (height_m ** 2)
    return round(bmi, 2)

In [9]:
health_recommender = AssistantAgent(
    name="health_recommender",
    system_message="""
    You are the Health Recommender Agent in a Smart Health Assistance team workflow. You receive a user profile JSON from the data_collector agent, which includes:
    - weight (kg)
    - height (cm)
    - age
    - gender
    - dietary_preference
    - any additional_information (optional)
    
    Your responsibilities are:
    1. Use the calculate_bmi tool to compute the user's BMI based on weight and height.
    2. Reflect on the BMI result and classify it into standard BMI categories:
       - BMI < 18.5: underweight
       - 18.5 ≤ BMI < 24.9: normal weight
       - 25 ≤ BMI < 29.9: overweight
       - BMI ≥ 30: obese
    3. Provide scientific-grounded, tecnical, detailed and professional health recommendations tailored to the user’s profile 
    (age, gender, dietary preference, and health conditions and goals iff provided) but directed to your team. Make it actionable and balanced.
    This information is going to be used by another two agents in charge of creating a meal plan and a workout schedule.
    4. Relay a structured JSON result to the next agent (the Diet Planner), containing at minimum:
       {
         "bmi": <value>,
         "bmi_category": "<category>",
         "recommendations": "<advice text>"
       }
    5. Do not proceed until you’ve generated the full JSON and health advice. End your response with the JSON block only, with no extra text after.
    Follow this template:
        {
            <json with bmi, bmi_category, and recommendations>
        }
    
    Ensure clarity, correctness, and completeness in your responses.
    """,
    model_client=openai_client,
    tools=[calculate_bmi],
    reflect_on_tool_use=True,
)

In [10]:
from autogen_core import CancellationToken
from autogen_agentchat.messages import TextMessage


async def health_recommendation(data_collected) -> dict:
    profile_str = json.dumps(data_collected, indent=2)
    response = await health_recommender.on_messages([TextMessage(content=profile_str, source="user")], CancellationToken())
    response_text = response.chat_message.to_text()
    print(f"[blue]🤖 {response_text}[/blue]")
    return response_text

In [11]:
health_recommendations = await health_recommendation(user_info)

🤖 {
  "bmi": 22.78,
  "bmi_category": "normal weight",
  "recommendations": "BastiMax, with a BMI of 22.78, falls within the normal weight category, which is excellent 
for your age and height. Given your goal to build muscle and your good fitness level, consider a structured workout
plan focusing on resistance training using the weights you have at home. Target major muscle groups with compound 
exercises like squats, deadlifts, bench presses, and bent-over rows, aiming for 3-4 sets of 8-12 reps for each 
exercise. Ensure a high-protein diet to support muscle repair and growth, incorporating sources such as chicken, 
fish, eggs, legumes, and dairy. Stay hydrated and consider integrating rest days into your routine to allow for 
muscle recovery. Since you have no dietary restrictions, include a balance of carbohydrates and healthy fats to 
fuel your workouts."
}

In [12]:
print_json(health_recommendations)

{
  "bmi": 22.78,
  "bmi_category": "normal weight",
  "recommendations": "BastiMax, with a BMI of 22.78, falls within the normal weight category, which is excellent for your age and height. Given your goal to build muscle and your good fitness level, consider a structured workout plan focusing on resistance training using the weights you have at home. Target major muscle groups with compound exercises like squats, deadlifts, bench presses, and bent-over rows, aiming for 3-4 sets of 8-12 reps for each exercise. Ensure a high-protein diet to support muscle repair and growth, incorporating sources such as chicken, fish, eggs, legumes, and dairy. Stay hydrated and consider integrating rest days into your routine to allow for muscle recovery. Since you have no dietary restrictions, include a balance of carbohydrates and healthy fats to fuel your workouts."
}

### Meal Plan

In [13]:
# Take profile and recommendations dicts, aggregate (opc), make a meal plan

meal_planner = AssistantAgent(
    name="meal_planner",
    system_message="""
    You are the Meal Planner in a multi-agent system for Smart Health Assistance.
    
    Your role is to take a user’s profile (including age, weight, height, gender, and dietary preferences), along with personalized recommendations (such as BMI score and health recommendations), and design a detailed daily meal plan that supports their health improvement journey.
    
    You DO NOT speak directly to the user — your role is purely functional and internal to the system. Your only output should be a well-structured meal plan in **markdown** format.
    
    Guidelines:
    - Tailor the meal plan to the user's **dietary preferences** (e.g., vegan, vegetarian, non-veg).
    - Consider **health recommendatinos and user information (goals) and BMI score**: For example: for weight loss, suggest calorie-conscious meals; for muscle gain, add protein-rich options; for diabetes or other conditions, offer suitable substitutions.
    - Be realistic, simple, and achievable — avoid obscure ingredients or complex cooking.
    - Ensure each meal is **nutritionally balanced**.
    - Output ONLY a markdown formatted meal plan with **headings and bullet points**.
    
    Structure your output like this:
    
    # Daily Meal Plan
    
    ## Breakfast
    - Example food item
    - ...
    
    ## Lunch
    - ...
    
    ## Snack
    - ...
    
    ## Dinner
    - ...
    
    Do not include explanations or wrap your output in code blocks. Provide at least one alternative to each meal.
    """,
    model_client=openai_client,
)

In [14]:
async def meal_planning(data_collected, health_recommendations) -> dict:
    profile_str = json.dumps(data_collected, indent=2)
    health_recomm_str = json.dumps(health_recommendations, indent=2)
    response = await meal_planner.on_messages([
        TextMessage(content=profile_str, source="user"),
        TextMessage(content=health_recomm_str, source="user"),
    ], CancellationToken())
    response_text = response.chat_message.to_text()
    print(f"[blue]🤖 {response_text}[/blue]")
    return response_text

In [15]:
meal_plan = await meal_planning(user_info, health_recommendations)

🤖 # Daily Meal Plan

## Breakfast
- Scrambled eggs with spinach and feta cheese
- Whole-grain toast with avocado

## Lunch
- Grilled chicken breast with quinoa and steamed broccoli
- Chickpea salad with cherry tomatoes, cucumber, and olive oil dressing

## Snack
- Greek yogurt with honey and mixed berries
- Protein smoothie with banana, spinach, and almond milk

## Dinner
- Baked salmon with sweet potato and asparagus
- Stir-fried tofu with mixed vegetables and brown rice

In [16]:
Markdown(meal_plan)

# Daily Meal Plan

## Breakfast
- Scrambled eggs with spinach and feta cheese
- Whole-grain toast with avocado

## Lunch
- Grilled chicken breast with quinoa and steamed broccoli
- Chickpea salad with cherry tomatoes, cucumber, and olive oil dressing

## Snack
- Greek yogurt with honey and mixed berries
- Protein smoothie with banana, spinach, and almond milk

## Dinner
- Baked salmon with sweet potato and asparagus
- Stir-fried tofu with mixed vegetables and brown rice

### Workout Schedule

In [17]:
workout_scheduler = AssistantAgent(
    name="meal_planner",
    system_message="""
    You are the Workout Scheduler in a multi-agent system for Smart Health Assistance.

    Your job is to generate a simple yet detailed, personalized workout routine based on:
    - The user's profile (age, gender, weight, height, dietary preference, etc.)
    - Their BMI and health goals (e.g., weight loss, strength gain, improved mobility)
    - Any health recommendations or constraints (e.g., weak knee, diabetes)
    - Their meal plan, to ensure workouts are appropriate for their energy and nutrition levels.
    
    You do NOT interact with users directly. Your output is intended for internal use and must be cleanly formatted in **markdown**.
    
    Guidelines:
    - Take into account user **fitness level**, **age**, and **physical limitations**.
    - If the user has any injuries or conditions, **adapt the routine** accordingly.
    - Focus on **achievability and safety**: do not overwhelm the user.
    - Include a **balanced mix** of cardio, strength, flexibility, and rest.
    - Structure the plan for **one week**, day-by-day (Mon–Sun), assuming beginner to moderate level unless otherwise specified.
    - Keep sessions to a **reasonable duration** (20–45 min).
    - Name each day, and break down each day's routine into steps or exercises with optional sets/reps/times.
    - Ouput two versions of your workout plan, one weight-assistance-based, and one body-weight-based.
    
    Your response must ONLY include a markdown workout schedule.
    Do not include any explanations, motivation, or wrap your response in code blocks.
""",
    model_client=openai_client,
)

In [18]:
async def design_workout(data_collected, health_recommendations, meal_plan) -> dict:
    profile_str = json.dumps(data_collected, indent=2)
    health_recomm_str = json.dumps(health_recommendations, indent=2)
    response = await workout_scheduler.on_messages([
        TextMessage(content=profile_str, source="user"),
        TextMessage(content=health_recomm_str, source="user"),
        TextMessage(content=meal_plan, source="user"),
    ], CancellationToken())
    response_text = response.chat_message.to_text()
    print(f"[blue]🤖 {response_text}[/blue]")
    return response_text

In [19]:
workout_schedule = await design_workout(user_info, health_recommendations, meal_plan)

🤖 # Workout Schedule for BastiMax

## Weight-Assistance-Based Plan

### Monday: Chest & Triceps
- Bench Press - 4 sets of 8-12 reps
- Dumbbell Flyes - 3 sets of 10-12 reps
- Tricep Dips - 3 sets of 10-15 reps
- Push-Ups (weighted if possible) - 3 sets of 8-12 reps

### Tuesday: Legs
- Goblet Squats - 4 sets of 8-12 reps
- Lunges (weighted) - 3 sets of 10-12 reps per leg
- Deadlifts - 4 sets of 8-12 reps
- Calf Raises - 3 sets of 12-15 reps

### Wednesday: Back & Biceps
- Bent-Over Rows - 4 sets of 8-12 reps
- Pull-Ups (assisted if necessary) - 3 sets of 5-8 reps
- Bicep Curls - 3 sets of 10-12 reps
- Face Pulls (if equipment available) - 3 sets of 12-15 reps

### Thursday: Active Recovery
- Light walking or stretching - 20-30 minutes
- Focus on flexibility exercises (e.g., seated toe touch, shoulder stretches)

### Friday: Full Body
- Kettlebell Swings - 3 sets of 10-15 reps
- Push Press - 3 sets of 8-12 reps
- Plank with Dumbbell Row - 3 sets of 8-10 reps per arm
- Burpees (modify if needed) - 3 sets of 5-8 reps

### Saturday: Core & Flexibility
- Plank - 3 sets of 30-45 seconds
- Russian Twists (with weight) - 3 sets of 10-12 reps per side
- Leg Raises - 3 sets of 10-15 reps
- Yoga or stretching session - 20-30 minutes

### Sunday: Rest Day
- Rest and recovery; hydrate and focus on nutrition

---

## Body-Weight-Based Plan

### Monday: Chest & Triceps
- Standard Push-Ups - 4 sets of 8-12 reps
- Decline Push-Ups (feet elevated) - 3 sets of 8-10 reps
- Tricep Dips (using a chair) - 3 sets of 10-15 reps
- Diamond Push-Ups - 3 sets of 5-8 reps

### Tuesday: Legs
- Bodyweight Squats - 4 sets of 12-15 reps
- Step-Ups (on a sturdy chair or step) - 3 sets of 10-12 reps per leg
- Glute Bridges - 3 sets of 12-15 reps
- Calf Raises - 3 sets of 15-20 reps

### Wednesday: Back & Biceps
- Inverted Rows (under a sturdy table) - 4 sets of 8-12 reps
- Close-Grip Push-Ups - 3 sets of 8-10 reps
- Plank to Push-Up Position - 3 sets of 5-8 reps
- Supermans - 3 sets of 10-15 reps

### Thursday: Active Recovery
- Light walking or stretching - 20-30 minutes
- Focus on flexibility exercises (e.g., seated toe touch, shoulder stretches)

### Friday: Full Body
- Burpees (modify if needed) - 3 sets of 5-8 reps
- Mountain Climbers - 3 sets of 30 seconds
- Plank with Shoulder Taps - 3 sets of 10-12 reps per side
- Jump Squats - 3 sets of 10-12 reps

### Saturday: Core & Flexibility
- Side Plank - 3 sets of 30 seconds per side
- Bicycle Crunches - 3 sets of 15-20 reps
- Plank - 3 sets of 30-45 seconds
- Yoga or stretching session - 20-30 minutes

### Sunday: Rest Day
- Rest and recovery; hydrate and focus on nutrition

In [20]:
Markdown(workout_schedule)

# Workout Schedule for BastiMax

## Weight-Assistance-Based Plan

### Monday: Chest & Triceps
- Bench Press - 4 sets of 8-12 reps
- Dumbbell Flyes - 3 sets of 10-12 reps
- Tricep Dips - 3 sets of 10-15 reps
- Push-Ups (weighted if possible) - 3 sets of 8-12 reps

### Tuesday: Legs
- Goblet Squats - 4 sets of 8-12 reps
- Lunges (weighted) - 3 sets of 10-12 reps per leg
- Deadlifts - 4 sets of 8-12 reps
- Calf Raises - 3 sets of 12-15 reps

### Wednesday: Back & Biceps
- Bent-Over Rows - 4 sets of 8-12 reps
- Pull-Ups (assisted if necessary) - 3 sets of 5-8 reps
- Bicep Curls - 3 sets of 10-12 reps
- Face Pulls (if equipment available) - 3 sets of 12-15 reps

### Thursday: Active Recovery
- Light walking or stretching - 20-30 minutes
- Focus on flexibility exercises (e.g., seated toe touch, shoulder stretches)

### Friday: Full Body
- Kettlebell Swings - 3 sets of 10-15 reps
- Push Press - 3 sets of 8-12 reps
- Plank with Dumbbell Row - 3 sets of 8-10 reps per arm
- Burpees (modify if needed) - 3 sets of 5-8 reps

### Saturday: Core & Flexibility
- Plank - 3 sets of 30-45 seconds
- Russian Twists (with weight) - 3 sets of 10-12 reps per side
- Leg Raises - 3 sets of 10-15 reps
- Yoga or stretching session - 20-30 minutes

### Sunday: Rest Day
- Rest and recovery; hydrate and focus on nutrition

---

## Body-Weight-Based Plan

### Monday: Chest & Triceps
- Standard Push-Ups - 4 sets of 8-12 reps
- Decline Push-Ups (feet elevated) - 3 sets of 8-10 reps
- Tricep Dips (using a chair) - 3 sets of 10-15 reps
- Diamond Push-Ups - 3 sets of 5-8 reps

### Tuesday: Legs
- Bodyweight Squats - 4 sets of 12-15 reps
- Step-Ups (on a sturdy chair or step) - 3 sets of 10-12 reps per leg
- Glute Bridges - 3 sets of 12-15 reps
- Calf Raises - 3 sets of 15-20 reps

### Wednesday: Back & Biceps
- Inverted Rows (under a sturdy table) - 4 sets of 8-12 reps
- Close-Grip Push-Ups - 3 sets of 8-10 reps
- Plank to Push-Up Position - 3 sets of 5-8 reps
- Supermans - 3 sets of 10-15 reps

### Thursday: Active Recovery
- Light walking or stretching - 20-30 minutes
- Focus on flexibility exercises (e.g., seated toe touch, shoulder stretches)

### Friday: Full Body
- Burpees (modify if needed) - 3 sets of 5-8 reps
- Mountain Climbers - 3 sets of 30 seconds
- Plank with Shoulder Taps - 3 sets of 10-12 reps per side
- Jump Squats - 3 sets of 10-12 reps

### Saturday: Core & Flexibility
- Side Plank - 3 sets of 30 seconds per side
- Bicycle Crunches - 3 sets of 15-20 reps
- Plank - 3 sets of 30-45 seconds
- Yoga or stretching session - 20-30 minutes

### Sunday: Rest Day
- Rest and recovery; hydrate and focus on nutrition

### Personalized Wellness Report 

In [21]:
reporter = AssistantAgent(
    name="reporter",
    system_message="""
    You are the final agent in a team working on Smart Health Assistance.

    Your task is to compile all the information received from previous agents into a well-structured and user-friendly Markdown report. 
    This report is intended for the end user and should clearly present their health profile, BMI analysis, personalized health recommendations, meal plan, and workout schedule.
    
    The tone should be encouraging, clear, and professional. Focus on clarity, formatting, and a personalized feel.
    
    Here is the expected structure of the report you must generate (You can refactor to fit the users profile/needs/interest):
    
    # 🧾 Your Personalized Wellness Report
    
    ## 👤 Profile Summary
    Summarize the user's key data: age, gender, weight, height, dietary preference, and any additional information (like goals or health conditions).
    
    ## 📊 BMI Analysis
    Include BMI score and category (e.g., Normal, Overweight) and explain what it means.
    
    ## 🩺 Health Recommendations
    Summarize actionable lifestyle or health advice from the analysis.
    
    ## 🍽️ Meal Plan
    Present a sample meal plan in a clean format (e.g., breakfast, lunch, dinner, snacks).
    
    ## 🏋️ Workout Schedule
    Include a weekly workout plan with intensity, type of exercise, and notes if needed.
    
    ---
    
    Ensure the formatting uses markdown headers, bullet points, and other features to improve readability.
    
    Return only the complete report in Markdown format.
    Do not include anything outside of the report or comment on the task.
    """,
    model_client=openai_client,
    )

In [22]:
async def create_wellness_plan_overview(data_collected, health_recommendations, meal_plan, workout_schedule) -> dict:
    profile_str = json.dumps(data_collected, indent=2)
    health_recomm_str = json.dumps(health_recommendations, indent=2)
    response = await reporter.on_messages([
        TextMessage(content=profile_str, source="user"),
        TextMessage(content=health_recomm_str, source="user"),
        TextMessage(content=meal_plan, source="user"),
        TextMessage(content=workout_schedule, source="user"),
    ], CancellationToken())
    response_text = response.chat_message.to_text()
    print(f"[blue]🤖 {response_text}[/blue]")
    return response_text

In [23]:
wellness_plan_summary = await create_wellness_plan_overview(user_info, health_recommendations, meal_plan, workout_schedule)

🤖 # 🧾 Your Personalized Wellness Report

## 👤 Profile Summary
- **Name:** BastiMax
- **Age:** 33
- **Gender:** Male
- **Weight:** 73 kg
- **Height:** 179 cm
- **Dietary Preference:** None
- **Goals:** Build muscle
- **Fitness Level:** Good
- **Workout Environment:** Home
- **Equipment Available:** Weights

## 📊 BMI Analysis
- **BMI Score:** 22.78
- **Category:** Normal Weight

Your BMI of 22.78 falls within the normal weight category, which is excellent for your age and height. This 
indicates that you're at a healthy weight, allowing you to effectively pursue your muscle-building goals.

## 🩺 Health Recommendations
BastiMax, to support your muscle-building efforts and maintain your overall health, consider the following 
recommendations:
- **Structured Workout Plan:** Focus on resistance training using the weights you have at home. Target major muscle
groups with compound exercises.
- **Dietary Focus:** Include a high-protein diet to support muscle repair and growth. Sources can include chicken, 
fish, eggs, legumes, and dairy.
- **Hydration:** Stay adequately hydrated throughout the day.
- **Rest Days:** Incorporate rest days into your routine to allow for muscle recovery.
- **Balanced Nutrition:** Since you have no dietary restrictions, maintain a balance of carbohydrates and healthy 
fats to fuel your workouts.

## 🍽️ Meal Plan
Here’s a sample meal plan tailored to support muscle building:

### Breakfast
- Scrambled eggs with spinach and feta cheese
- Whole-grain toast with avocado

### Lunch
- Grilled chicken breast with quinoa and steamed broccoli
- Chickpea salad with cherry tomatoes, cucumber, and olive oil dressing

### Snack
- Greek yogurt with honey and mixed berries
- Protein smoothie with banana, spinach, and almond milk

### Dinner
- Baked salmon with sweet potato and asparagus
- Stir-fried tofu with mixed vegetables and brown rice

## 🏋️ Workout Schedule

### Weight-Assistance-Based Plan

#### Monday: Chest & Triceps
- Bench Press - 4 sets of 8-12 reps
- Dumbbell Flyes - 3 sets of 10-12 reps
- Tricep Dips - 3 sets of 10-15 reps
- Push-Ups (weighted if possible) - 3 sets of 8-12 reps

#### Tuesday: Legs
- Goblet Squats - 4 sets of 8-12 reps
- Lunges (weighted) - 3 sets of 10-12 reps per leg
- Deadlifts - 4 sets of 8-12 reps
- Calf Raises - 3 sets of 12-15 reps

#### Wednesday: Back & Biceps
- Bent-Over Rows - 4 sets of 8-12 reps
- Pull-Ups (assisted if necessary) - 3 sets of 5-8 reps
- Bicep Curls - 3 sets of 10-12 reps
- Face Pulls (if equipment available) - 3 sets of 12-15 reps

#### Thursday: Active Recovery
- Light walking or stretching - 20-30 minutes
- Focus on flexibility exercises (e.g., seated toe touch, shoulder stretches)

#### Friday: Full Body
- Kettlebell Swings - 3 sets of 10-15 reps
- Push Press - 3 sets of 8-12 reps
- Plank with Dumbbell Row - 3 sets of 8-10 reps per arm
- Burpees (modify if needed) - 3 sets of 5-8 reps

#### Saturday: Core & Flexibility
- Plank - 3 sets of 30-45 seconds
- Russian Twists (with weight) - 3 sets of 10-12 reps per side
- Leg Raises - 3 sets of 10-15 reps
- Yoga or stretching session - 20-30 minutes

#### Sunday: Rest Day
- Rest and recovery; hydrate and focus on nutrition

---

You are on a promising path to achieve your goals. Stay committed, and you'll see great results!

In [24]:
Markdown(wellness_plan_summary)

# 🧾 Your Personalized Wellness Report

## 👤 Profile Summary
- **Name:** BastiMax
- **Age:** 33
- **Gender:** Male
- **Weight:** 73 kg
- **Height:** 179 cm
- **Dietary Preference:** None
- **Goals:** Build muscle
- **Fitness Level:** Good
- **Workout Environment:** Home
- **Equipment Available:** Weights

## 📊 BMI Analysis
- **BMI Score:** 22.78
- **Category:** Normal Weight

Your BMI of 22.78 falls within the normal weight category, which is excellent for your age and height. This indicates that you're at a healthy weight, allowing you to effectively pursue your muscle-building goals.

## 🩺 Health Recommendations
BastiMax, to support your muscle-building efforts and maintain your overall health, consider the following recommendations:
- **Structured Workout Plan:** Focus on resistance training using the weights you have at home. Target major muscle groups with compound exercises.
- **Dietary Focus:** Include a high-protein diet to support muscle repair and growth. Sources can include chicken, fish, eggs, legumes, and dairy.
- **Hydration:** Stay adequately hydrated throughout the day.
- **Rest Days:** Incorporate rest days into your routine to allow for muscle recovery.
- **Balanced Nutrition:** Since you have no dietary restrictions, maintain a balance of carbohydrates and healthy fats to fuel your workouts.

## 🍽️ Meal Plan
Here’s a sample meal plan tailored to support muscle building:

### Breakfast
- Scrambled eggs with spinach and feta cheese
- Whole-grain toast with avocado

### Lunch
- Grilled chicken breast with quinoa and steamed broccoli
- Chickpea salad with cherry tomatoes, cucumber, and olive oil dressing

### Snack
- Greek yogurt with honey and mixed berries
- Protein smoothie with banana, spinach, and almond milk

### Dinner
- Baked salmon with sweet potato and asparagus
- Stir-fried tofu with mixed vegetables and brown rice

## 🏋️ Workout Schedule

### Weight-Assistance-Based Plan

#### Monday: Chest & Triceps
- Bench Press - 4 sets of 8-12 reps
- Dumbbell Flyes - 3 sets of 10-12 reps
- Tricep Dips - 3 sets of 10-15 reps
- Push-Ups (weighted if possible) - 3 sets of 8-12 reps

#### Tuesday: Legs
- Goblet Squats - 4 sets of 8-12 reps
- Lunges (weighted) - 3 sets of 10-12 reps per leg
- Deadlifts - 4 sets of 8-12 reps
- Calf Raises - 3 sets of 12-15 reps

#### Wednesday: Back & Biceps
- Bent-Over Rows - 4 sets of 8-12 reps
- Pull-Ups (assisted if necessary) - 3 sets of 5-8 reps
- Bicep Curls - 3 sets of 10-12 reps
- Face Pulls (if equipment available) - 3 sets of 12-15 reps

#### Thursday: Active Recovery
- Light walking or stretching - 20-30 minutes
- Focus on flexibility exercises (e.g., seated toe touch, shoulder stretches)

#### Friday: Full Body
- Kettlebell Swings - 3 sets of 10-15 reps
- Push Press - 3 sets of 8-12 reps
- Plank with Dumbbell Row - 3 sets of 8-10 reps per arm
- Burpees (modify if needed) - 3 sets of 5-8 reps

#### Saturday: Core & Flexibility
- Plank - 3 sets of 30-45 seconds
- Russian Twists (with weight) - 3 sets of 10-12 reps per side
- Leg Raises - 3 sets of 10-15 reps
- Yoga or stretching session - 20-30 minutes

#### Sunday: Rest Day
- Rest and recovery; hydrate and focus on nutrition

---

You are on a promising path to achieve your goals. Stay committed, and you'll see great results!

### Markdown to HTML with Antropic

In [56]:
from autogen_ext.models.anthropic import AnthropicChatCompletionClient

anthropic_client = AnthropicChatCompletionClient(
    model="claude-opus-4-20250514",
    max_tokens=8000,
    temperature=0.3,
)

In [57]:
report_html_generator = AssistantAgent(
    name="report_html_generator",
    system_message=f"""
        You are a professional health dashboard web designer. Convert the following Markdown health report into a **modern, visually engaging, and responsive HTML page**.
        You are a health-focused web designer.
        
        Convert the following Markdown report into a **complete, visually appealing, responsive HTML page**:
        - Render all tables fully (do not truncate or skip days).
        - Include semantic HTML, inline styles, and graphs where appropriate.
        - You must make a week calendar with the workout schedule/plan. Make sure all the workout details are properly displayed. This is critical.
        - Enhance layout clarity with sections and headers.
        - Use icons or subtle visuals to improve appeal.
        - The page should feel like a polished health dashboard for end users.
        
        Markdown Content:
        
        {wellness_plan_summary}
        
        Instructions:
        - Add **header banners, colors, and visual hierarchy** for sections (e.g., bold headers, subheaders, separators).
        - **Convert health metrics** (e.g., BMI, meal plans, workout schedules) into **semantic layouts** (e.g., tables, lists, blocks).
        - If appropriate, **embed graphs** (simple bar/line charts using inline SVG or styled divs), especially for tracking health progress or nutrition breakdown.
        - Make the page **mobile-friendly and easy to read**.
        - Use **semantic HTML5**: `<header>`, `<section>`, `<article>`, `<footer>`, etc.
        - Add minimal but elegant **inline CSS styles** for layout and visual appeal (no external stylesheets or JS).
        - Include a proper `<title>` tag and a `<meta charset="UTF-8">`.
        - Ensure special characters are escaped and valid.
        - Output only the **full HTML document** — no comments or explanations.
        
        Final HTML should look professional and user-friendly.
        """,
    model_client=anthropic_client,
)

In [58]:
async def convert_report_to_html(markdown_text: str, filename: str = "final_report.html") -> str:
    messages = [TextMessage(content=markdown_text, source="user")]
    response = await report_html_generator.on_messages(messages, CancellationToken())
    html = response.chat_message.to_text()

    with open(filename, "w", encoding="utf-8") as f:
        f.write(html)

    print(f"[✅] HTML report saved to: {filename}")
    return html

In [59]:
report_html = await convert_report_to_html(wellness_plan_summary, "sebastian_wellness_report.html")

[✅] HTML report saved to: sebastian_wellness_report.html

In [60]:
from IPython.core.display import HTML
HTML(report_html)

## Workflow

In [81]:
async def enrich_with_health_recs(user_info):
    return {
        "user_info": user_info,
        "health_recommendations": await health_recommendation(user_info)
    }

async def enrich_with_meal_plan(data):
    return {
        **data,
        "meal_plan": await meal_planning(data["user_info"], data["health_recommendations"])
    }

async def enrich_with_workout_schedule(data):
    return {
        **data,
        "workout_schedule": await design_workout(data["user_info"], data["health_recommendations"], data["meal_plan"])
    }

async def generate_report(data):
    return await create_wellness_plan_overview(
        data["user_info"], data["health_recommendations"], data["meal_plan"], data["workout_schedule"]
    )

collect_data_chain = RunnableLambda(collect_data)
health_recomm_chain = RunnableLambda(enrich_with_health_recs)
meal_plan_chain = RunnableLambda(enrich_with_meal_plan)
workout_plan_chain = RunnableLambda(enrich_with_workout_schedule)
report_chain = RunnableLambda(generate_report)
html_chain = RunnableLambda(convert_report_to_html)

full_chain = (
    collect_data_chain
    | health_recomm_chain
    | meal_plan_chain
    | workout_plan_chain
    | report_chain
    | html_chain
)

result = await full_chain.ainvoke(None)

Enter your response:  Papi


Papi

🤖 Great! I’ll format your name as "Papi."

Now, could you please provide me with your weight in kilograms?

Enter your response:  75


75

...

🤖 # Daily Meal Plan

## Breakfast
- Oatmeal topped with sliced banana and a sprinkle of cinnamon
- Smoothie with spinach, mixed berries, and protein powder

## Lunch
- Turkey and avocado wrap in a whole grain tortilla with spinach and tomatoes
- Quinoa salad with black beans, corn, bell peppers, and lime dressing

## Snack
- Apple slices with almond butter
- Cottage cheese with pineapple chunks

## Dinner
- Grilled chicken breast with brown rice and steamed mixed vegetables
- Baked tilapia with quinoa and a side salad of mixed greens with olive oil vinaigrette

🤖 # Workout Schedule for Papi

## Weight-Assistance-Based Plan

### Monday: Chest & Triceps
- Bench Press - 3 sets of 8-12 reps
- Incline Dumbbell Press - 3 sets of 8-12 reps
- Tricep Pressdown (using cable or resistance band) - 3 sets of 10-15 reps
- Push-Ups - 3 sets of 8-12 reps

### Tuesday: Legs
- Barbell Squats - 4 sets of 8-12 reps
- Romanian Deadlifts - 3 sets of 8-12 reps
- Lunges (weighted) - 3 sets of 10-12 reps per leg
- Leg Press (if available) - 3 sets of 10-12 reps

### Wednesday: Back & Biceps
- Bent-Over Rows - 4 sets of 8-12 reps
- Pull-Ups (assisted if necessary) - 3 sets of 5-8 reps
- Bicep Curls - 3 sets of 10-12 reps
- Seated Rows (machine) - 3 sets of 10-12 reps

### Thursday: Active Recovery
- Light walking or dynamic stretching - 20-30 minutes
- Focus on flexibility exercises (e.g., quadriceps stretch, hamstring stretch)

### Friday: Full Body
- Kettlebell Swings - 3 sets of 10-15 reps
- Dumbbell Thrusters - 3 sets of 8-12 reps
- Plank with Dumbbell Row - 3 sets of 8-10 reps per side
- Burpees - 3 sets of 5-8 reps

### Saturday: Core & Flexibility
- Plank - 3 sets of 30-45 seconds
- Hanging Leg Raises - 3 sets of 10-15 reps
- Russian Twists (with weight) - 3 sets of 10-12 reps per side
- Yoga session or stretching - 20-30 minutes

### Sunday: Rest Day
- Rest and recovery; focus on hydration and nutrition

---

## Body-Weight-Based Plan

### Monday: Chest & Triceps
- Standard Push-Ups - 3 sets of 8-12 reps
- Incline Push-Ups (hands elevated) - 3 sets of 8-12 reps
- Tricep Dips (using a chair) - 3 sets of 10-15 reps
- Diamond Push-Ups - 3 sets of 5-8 reps

### Tuesday: Legs
- Bodyweight Squats - 4 sets of 12-15 reps
- Step-Ups (onto a sturdy chair/step) - 3 sets of 10-12 reps per leg
- Walking Lunges - 3 sets of 10-12 reps per leg
- Glute Bridges - 3 sets of 12-15 reps

### Wednesday: Back & Biceps
- Inverted Rows (using a sturdy table) - 4 sets of 8-12 reps
- Close-Grip Push-Ups - 3 sets of 8-10 reps
- Plank Up-Downs - 3 sets of 5-8 reps
- Supermans - 3 sets of 10-15 reps

### Thursday: Active Recovery
- Light walking or dynamic stretching - 20-30 minutes
- Focus on flexibility exercises (e.g., quadriceps stretch, hamstring stretch)

### Friday: Full Body
- Burpees (modify if needed) - 3 sets of 5-8 reps
- Mountain Climbers - 3 sets of 30 seconds
- Plank with Shoulder Taps - 3 sets of 10-12 reps per side
- Squat Jumps - 3 sets of 10-12 reps

### Saturday: Core & Flexibility
- Side Plank - 3 sets of 30 seconds per side
- Bicycle Crunches - 3 sets of 15-20 reps
- Plank - 3 sets of 30-45 seconds
- Yoga session or stretching - 20-30 minutes

### Sunday: Rest Day
- Rest and recovery; focus on hydration and nutrition

C:\Users\lavml\Documents\GitHub\Alfredo\.venv\Lib\site-packages\rich\text.py:1131: RuntimeWarning: coroutine 'health_recommendation' was never awaited
  _Text(


🤖 # 🧾 Your Personalized Wellness Report

## 👤 Profile Summary
- **Name:** Papi
- **Age:** 31
- **Gender:** Male
- **Weight:** 75 kg
- **Height:** 189 cm
- **Dietary Preference:** None

## 📊 BMI Analysis
- **BMI Score:** 21.0
- **Category:** Normal Weight

Your BMI of 21.0 categorizes you as having a normal weight, indicating that you maintain a healthy body weight 
relative to your height and age. This foundation supports optimal health and fitness.

## 🩺 Health Recommendations
To maintain your weight and support overall health, here are some tips:
- **Balanced Diet:** Focus on a diverse diet with ample fruits, vegetables, whole grains, lean proteins, and 
healthy fats.
- **Physical Activity:** Engage in at least 150 minutes of moderate aerobic exercise each week, plus 
muscle-strengthening activities on two or more days.
- **Maintain Muscle Mass:** Regular exercise is key to supporting muscle mass and promoting overall metabolic 
health and well-being.

## 🍽️ Meal Plan
Here's a sample meal plan to help you maintain a balanced diet:

### Breakfast
- Oatmeal topped with sliced banana and a sprinkle of cinnamon
- Smoothie with spinach, mixed berries, and protein powder

### Lunch
- Turkey and avocado wrap in a whole grain tortilla with spinach and tomatoes
- Quinoa salad with black beans, corn, bell peppers, and lime dressing

### Snack
- Apple slices with almond butter
- Cottage cheese with pineapple chunks

### Dinner
- Grilled chicken breast with brown rice and steamed mixed vegetables
- Baked tilapia with quinoa and a side salad of mixed greens with olive oil vinaigrette

## 🏋️ Workout Schedule

### Weight-Assistance-Based Plan

#### Monday: Chest & Triceps
- Bench Press - 3 sets of 8-12 reps
- Incline Dumbbell Press - 3 sets of 8-12 reps
- Tricep Pressdown (using cable or resistance band) - 3 sets of 10-15 reps
- Push-Ups - 3 sets of 8-12 reps

#### Tuesday: Legs
- Barbell Squats - 4 sets of 8-12 reps
- Romanian Deadlifts - 3 sets of 8-12 reps
- Lunges (weighted) - 3 sets of 10-12 reps per leg
- Leg Press (if available) - 3 sets of 10-12 reps

#### Wednesday: Back & Biceps
- Bent-Over Rows - 4 sets of 8-12 reps
- Pull-Ups (assisted if necessary) - 3 sets of 5-8 reps
- Bicep Curls - 3 sets of 10-12 reps
- Seated Rows (machine) - 3 sets of 10-12 reps

#### Thursday: Active Recovery
- Light walking or dynamic stretching - 20-30 minutes
- Focus on flexibility exercises (e.g., quadriceps stretch, hamstring stretch)

#### Friday: Full Body
- Kettlebell Swings - 3 sets of 10-15 reps
- Dumbbell Thrusters - 3 sets of 8-12 reps
- Plank with Dumbbell Row - 3 sets of 8-10 reps per side
- Burpees - 3 sets of 5-8 reps

#### Saturday: Core & Flexibility
- Plank - 3 sets of 30-45 seconds
- Hanging Leg Raises - 3 sets of 10-15 reps
- Russian Twists (with weight) - 3 sets of 10-12 reps per side
- Yoga session or stretching - 20-30 minutes

#### Sunday: Rest Day
- Rest and recovery; focus on hydration and nutrition

---

Keep up the great work, Papi! You're on the right track to maintaining your health and achieving your fitness 
goals. Stay consistent and motivated!

[✅] HTML report saved to: final_report.html